In [ ]:
#

**# 🐶End-toEnd multi-class Dog Breed Classification**
  
  This notebook builds an end-to-End image classifier using TensorFlow 2.0 and TensorFlow Hub.

## 1.Problem Statement
Identifying thweb breed of a dog an image of a dog
in Real time picking a pic of a dog and it identify its breed
## 2.Data
Data is from Kagle competition.
## 3.Evaluation
Evaluation Based on "Multi class Log Loss" Between the predicted probability and observed target
## 4.Features
Some inforamation about Data:
*We are dealing with images(unstructured data) so it's probability best we use seep learning/transfer learning
* there are around 120 breeds of dogs (it means 120 differnt classes)
*there are 10,000+ images in Training set with the labels
*there are 10,000+ images in Test set whose we need to predicts...


# Get out Workplace ready
*importing TensorFlow
*importing TensorFlow Hub
*Make sure qwe are using GPU

In [ ]:
#Importing neccesary tools in google colab
import tensorflow as tf
import tensorflow_hub as hub
print(tf.__version__)
print(hub.__version__)

#Cheking for GPU availability
print('GPU',"available (yesssss!!!!)"if tf.config.list_logical_devices('GPU')else 'not avail...print')

2.15.0
0.16.1
GPU available (yesssss!!!!)


In [25]:
# #Importing Label file...
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
labels=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/labels.csv')
submition_samle=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/sample_submission.csv')
labels['breed'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10222 entries, 0 to 10221
Series name: breed
Non-Null Count  Dtype 
--------------  ----- 
10222 non-null  object
dtypes: object(1)
memory usage: 80.0+ KB


## Getting our data ready (tuning into tensorflow)
With all machin learning model ,our data has to be in numarical formate. So thats what we'll be doing first.Turning our images into Tensors(numerical representations)..

*NOTE:all ML model take data as numerical form for computations..
     By which tool the data change is another matter...   
    Lets start by accessing out data and checking out the labels    

In [93]:
# #Importing Label data ...
import pandas as pd

labels=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/labels.csv')
submition_samle=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/sample_submission.csv')
labels.id[10221]
#labels['breed'].value_counts().plot.bar(figsize=(10,5))

'fff43b07992508bc822f33d8ffd902ae'

In [86]:
#Lets view an image
from IPython.display import Image
train_path='/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/train/'
id='00214f311d5d2247d5dfe4fe24b2303d.jpg'
#Image(train_path+id)

In [94]:
#Create pathname from ID's
filenames=[train_path + fname + '.jpg'  for fname in labels['id'] ]
filenames[10221]


'/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/train/fff43b07992508bc822f33d8ffd902ae.jpg'

In [99]:
#Check whether number of filenames matches number of actual image files
import os
if len(os.listdir('/content/drive/MyDrive/Colab Notebooks/Dog Vision/dog-breed-identification/train'))==len(filenames):
  print('Filenames match actual amount of files!! proceed')
else:
  print('Filenames do no match with actual amount of files!! check the target directory..')

Filenames do no match with actual amount of files!! check the target directory..


Since we are now got our training images filepaths in a list , let's prepare our labels

In [95]:
import numpy as np
labels=np.array(labels['breed'])
labels

array(['boston_bull', 'dingo', 'pekinese', ..., 'airedale',
       'miniature_pinscher', 'chesapeake_bay_retriever'], dtype=object)

In [98]:
#see the no of lables same as no of filenames..
len(labels)==len(filenames)
len(filenames)

10222

Number of labels matches numbers of filenames!

In [101]:
#Find the uniue labels values
unique_labels=np.unique(labels)
len(unique_labels)

120

In [124]:
label_boolean_array=[]
for lable in labels:
 label_boolean_array.append((lable==unique_labels))
label_boolean_array=np.array(label_boolean_array)
boolean_lables=[lable==unique_labels for label in labels]
boolean_lables


[array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]),
 array([False, False, False, False, False, Fal

In [125]:
labels_in_numricForm=[np.where(lb==unique_labels)[0][0] for lb in labels]
np.array(labels_in_numricForm)

[19,
 37,
 85,
 15,
 49,
 10,
 10,
 18,
 7,
 97,
 99,
 113,
 73,
 15,
 79,
 2,
 118,
 89,
 68,
 21,
 38,
 83,
 83,
 10,
 37,
 49,
 106,
 58,
 12,
 26,
 0,
 67,
 55,
 40,
 113,
 114,
 48,
 49,
 19,
 54,
 68,
 58,
 36,
 37,
 0,
 110,
 17,
 68,
 109,
 80,
 2,
 58,
 113,
 58,
 118,
 48,
 0,
 100,
 57,
 66,
 46,
 53,
 8,
 6,
 95,
 90,
 56,
 5,
 46,
 14,
 94,
 77,
 24,
 40,
 63,
 84,
 16,
 24,
 5,
 97,
 42,
 33,
 71,
 115,
 29,
 17,
 93,
 114,
 42,
 57,
 73,
 14,
 88,
 106,
 72,
 40,
 118,
 88,
 68,
 21,
 90,
 65,
 3,
 118,
 40,
 69,
 48,
 74,
 57,
 90,
 93,
 69,
 25,
 37,
 56,
 3,
 24,
 25,
 11,
 15,
 57,
 57,
 42,
 115,
 77,
 4,
 77,
 49,
 90,
 70,
 85,
 27,
 60,
 36,
 31,
 68,
 16,
 56,
 96,
 113,
 54,
 100,
 17,
 70,
 26,
 56,
 1,
 82,
 92,
 69,
 18,
 92,
 113,
 75,
 10,
 43,
 106,
 26,
 83,
 54,
 110,
 97,
 59,
 71,
 73,
 11,
 53,
 96,
 18,
 22,
 111,
 30,
 0,
 44,
 94,
 4,
 113,
 80,
 25,
 33,
 40,
 81,
 103,
 2,
 104,
 12,
 31,
 103,
 39,
 50,
 101,
 84,
 38,
 55,
 0,
 14,
 115,
 17,


In [118]:
#labels[0]==unique_labels
collab_data={'file_path':filenames,
             'labels_num_form':labels_in_numricForm}
collab_data_df=pd.DataFrame(collab_data)
collab_data_df.describe()


,labels_num_form
count,10222.000000
mean,59.400215
std,34.840374
min,0.000000
25%,29.000000
50%,60.000000
75%,90.000000
max,119.000000
